# 生成器

我们可以通过循环或列表生成式来创建一个很大的列表，但由于受到内存的限制，有时候我们无法把要处理的数据一次性加载到内存，让到List中，然后再处理。

尤其是一些流式计算的任务中，数据是online生成的，我们不可能预知数据整体规模。

如果一旦我们知道了数据生成的方式或数据读取的方式，我们就可以使用**生成器(generator)**来一边循环一边计算。

创建生成器的第一种方式法，直接采用和List生成式类型的语法，只是把`[]`改为`()`

In [6]:
g = (i * i for i in range(10))
g

<generator object <genexpr> at 0x7fa40c5a2570>

那么怎么读取生成器中的数据呢，两种方式，一种是使用`next()`来一个一个读取，一种是使用`for..in..`

In [7]:
next(g)

0

In [8]:
next(g)

1

In [9]:
next(g)

4

In [10]:
for i in g:
    print(i)

9
16
25
36
49
64
81


我们可以看到g中的数据一量被消费掉，就丢失了，所以`for..in`在读取时，是接着最后一次`next()`的。

当genenator生成完最后一个目标后，会产生一个`StopIteration`的异常。

In [11]:
next(g)

StopIteration: 

把以在实际中，我们很少真正会使用`next()`来读取生成器中的数据，而是会用`for..in..`，这样会自动的处理`StopIteration`异常。

创建generator的第二种方式是，含有yield的函数。

In [30]:
def fibonacci(n):
    a = 0
    b = 1
    i = 0
    while i < n:
        yield b
        a, b = b, a + b
        i  = i + 1

In [31]:
g = fibonacci(10)
g

<generator object fibonacci at 0x7fa40c4055c8>

这就是定义generator的另一种方法。如果一个函数定义中包含yield关键字，那么这个函数就不再是一个普通函数，而是一个generator。

genrator的生成是随着函数绑定一个具体的参数时生成的。普通的函数调用后产生的是一个返回结构，而generator函数调用后产生的是一个generator对象。

这里，最难理解的就是generator和函数的执行流程不一样。函数是顺序执行，遇到return语句或者最后一行函数语句就返回。而变成generator的函数，在每次调用next()的时候执行，遇到yield语句返回，再次执行时从上次返回的yield语句处继续执行。

下面的函数内部有三次yield，所以这个generator调用三次后，就会返回`StopIterator`

In [33]:
def odd():
    print('step 1')
    yield 1
    print('step 2')
    yield(3)
    print('step 3')
    yield(5)
g = odd()
next(g)
next(g)
next(g)
next(g)

step 1
step 2
step 3


StopIteration: 

# 可迭代对象和迭代器

可以直接作用于for循环的对象统称为可迭代对象：`Iterable`，这些对象包括了：`list`、`tuple`、`set`、`dict`、`str`，还包含上面我们介绍的`generator`。

可以使用`isinstance()`判断一个对象是否是`Iterable`对象。

而生成器不但可以作用于`for`循环，还可以被`next()`函数不断调用并返回下一个值，直到最后抛出`StopIteration`错误表示无法继续返回下一个值了。

可以被`next()`函数调用并不断返回下一个值的对象称为迭代器：`Iterator`。

可以使用`isinstance()`判断一个对象是否是`Iterator`对象：

In [36]:
from collections import Iterator
isinstance((x * x for x in range(10)), Iterator)

True

In [37]:
isinstance(fibonacci(5), Iterator)

True

In [38]:
isinstance([1, 2, 3, 4, 5], Iterator)

False

生成器都是`Iterator`对象，但`list`、`dict`、`str`虽然是`Iterable`，却不是`Iterator`。

把`list`、`dict`、`str`等`Iterable`变成`Iterator`可以使用`iter()`函数：

In [39]:
isinstance(iter([1, 2, 3, 4, 5]), Iterator)

True

你可能会问，为什么list、dict、str等数据类型不是Iterator？

这是因为Python的`Iterator`对象表示的是一个数据流，`Iterator`对象可以被`next()`函数调用并不断返回下一个数据，直到没有数据时抛出`StopIteration`错误。可以把这个数据流看做是一个有序序列，但我们却不能提前知道序列的长度，只能不断通过`next()`函数实现按需计算下一个数据，所以`Iterator`的计算是**惰性**的，只有在需要返回下一个数据时它才会计算。

`Iterator`甚至可以表示一个无限大的数据流，例如全体自然数。而使用`list`是永远不可能存储全体自然数的。

# 总结

- 凡是可作用于for循环的对象都是Iterable类型；
- 凡是可作用于next()函数的对象都是Iterator类型，它们表示一个惰性计算的序列；
- 集合数据类型如list、dict、str等是Iterable但不是Iterator，不过可以通过iter()函数获得一个Iterator对象。
- Python的for循环本质上就是通过不断调用next()函数实现的